<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','EQUITAS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

Mounted at /content/drive


In [3]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()

In [4]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_v20 = df_v20.append(df_tmp)

    return df_v20.sort_index()

In [5]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

In [11]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [12]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'V40N') | (df_v20_prospects['Criteria'] == 'V40')]

In [16]:
pd.concat([df_v20_prospects_V40N, df_v20_prospects_200SMA])

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2020-09-07 00:00:00+05:30,ASTRAZEN.NS,27.20,3421.0,4537.492676,V40N
2020-12-28 00:00:00+05:30,WHIRLPOOL.NS,24.03,2119.0,2566.310791,V40
2020-12-28 00:00:00+05:30,JCHAC.NS,30.12,2206.0,2635.550049,V40N
2021-02-02 00:00:00+05:30,INDIGOPNTS.NS,28.45,3112.0,3111.586914,V40N
2021-02-09 00:00:00+05:30,TEAMLEASE.NS,38.48,2299.0,3577.149902,V40N
2021-03-02 00:00:00+05:30,SYMPHONY.NS,40.43,896.0,1420.163330,V40N
2021-04-06 00:00:00+05:30,LALPATHLAB.NS,24.44,2105.0,3032.469238,V40N
2021-05-28 00:00:00+05:30,LUXIND.NS,61.36,1635.0,3244.103760,V40N
2021-07-14 00:00:00+05:30,5PAISA.NS,30.12,333.0,553.200012,V40N


In [15]:
df_v20_prospects['Criteria'].value_counts()

2T3Y    19
V40N    18
SS       2
V40      1
TBD      1
Name: Criteria, dtype: int64